<a href="https://colab.research.google.com/github/ArthurHSM/ml-ti-incident-classifier/blob/main/2_dumpModel_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [79]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import joblib
import re
from scipy.sparse import csr_matrix

# Variáveis de Deploy e Artefatos

In [80]:
# mapa de categorias salvo
try:
    OHE_CATEGORY_MAP = joblib.load("model_artifacts_deploy/ohe_category_map.pkl")
except FileNotFoundError:
    raise RuntimeError("Erro ao carregar artefatos. Verifique a pasta 'model_artifacts_deploy'.")

# Definições de Colunas
CATEGORICAL_COLS = ['source', 'environment', 'severity', 'metric_name', 'ci_tratado']
# O input da API ainda terá a coluna 'ci' original, mas a saída precisa de 'ci_tratado'
INPUT_COLS_API = ['source', 'environment', 'severity', 'metric_name', 'ci', 'maintenance']
THRESHOLD_NEGOCIO = 0.75

# OHE

In [81]:
# Instanciação do OHE (Repetido aqui para clareza da função final)
ohe_transformer = OneHotEncoder(
    categories=[OHE_CATEGORY_MAP[col] for col in CATEGORICAL_COLS],
    handle_unknown='ignore',
    sparse_output=False
)

dummy_data_for_fit = pd.DataFrame([
    [OHE_CATEGORY_MAP[col][0] for col in CATEGORICAL_COLS]
], columns=CATEGORICAL_COLS)
ohe_transformer.fit(dummy_data_for_fit.values)

OneHotEncoder(categories=[['Dynatrace', 'Zabbix', 'CloudWatch', 'ThousandEyes'],
                          ['prod', 'hml', 'dev', 'unknown'],
                          ['ok', 'warning', 'minor', 'major', 'critical',
                           'clear'],
                          ['http_response_time', 'disk_space_usage',
                           'cpu_usage', 'queue_length', 'connection_errors',
                           'memory_usage', 'swap_usage', 'thread_count',
                           'ping_failed', 'service_unavailable', 'packet_loss',
                           'disk_io', 'response_time', 'out_of_memory_kill',
                           'network_latency', 'db_connections'],
                          ['server', 'app', 'db', 'router', 'switch',
                           'UNKNOWN_CI']],
              handle_unknown='ignore', sparse_output=False)

# Main functions

In [72]:

def _process_ci_column(ci_value):
    """
    Função auxiliar para aplicar o tratamento do CI.
    1. Trata nulos/vazios.
    2. Extrai apenas as letras (ex: 'app-7' -> 'app').
    """
    # 1. Imputação de Nulos/Vazios
    if pd.isna(ci_value) or str(ci_value).strip() == '':
        return "unknown_ci"

    # 2. Extração de Apenas Letras (Ex: 'db-42' -> 'db')
    # O regex [a-zA-Z]+ encontra uma ou mais letras no início da string
    match = re.search(r'^[a-zA-Z]+', str(ci_value).lower())

    if match:
        return match.group(0) # Retorna a string de letras (ex: 'app')
    else:
        # Se não encontrar letras (ex: '404-error'), trata como desconhecido
        return "unknown_ci"


from scipy.sparse import csr_matrix, hstack
# (Assumindo que as funções auxiliares e variáveis globais estão disponíveis)

def prepare_alert_for_model(alert_data_dict: dict) -> csr_matrix:
    """
    Função principal que replica Feature Engineering para um único alerta.
    Retorna uma matriz esparsa (CSR).
    """

    # 1. Inicialização do DataFrame a partir do JSON (usa as colunas de entrada da API)
    df_alert = pd.DataFrame([alert_data_dict], columns=INPUT_COLS_API)

    # 2. ETAPA DE PRÉ-PROCESSAMENTO/TRANSFORMAÇÃO (Regras de Limpeza)

    # a. Criação da feature 'ci_tratado' (inclui imputação e limpeza)
    df_alert['ci_tratado'] = df_alert['ci'].apply(_process_ci_column)

    # b. Replicar a feature 'maintenance_int' (Binária)
    df_alert['maintenance_int'] = df_alert['maintenance'].apply(
        lambda x: 1 if str(x).lower() == 'true' else 0
    )

    # 3. ETAPA DE VETORIZAÇÃO (One-Hot Encoding e Concatenação)

    categorical_data = df_alert[CATEGORICAL_COLS]

    # Aplica o OHE (Transforma em matriz esparsa, usando .values)
    # Assumimos que ohe_transformer foi inicializado com sparse_output=True
    ohe_features = ohe_transformer.transform(categorical_data.values)

    # Converte a feature 'maintenance_int' (Densa) em uma matriz esparsa (CSR)
    maintenance_sparse = csr_matrix(df_alert[['maintenance_int']].values)

    # Concatena (Replica o VectorAssembler) usando hstack (Sparse Horizontal Stack)
    final_vector = hstack([
        ohe_features,
        maintenance_sparse
    ], format='csr')

    # 4. Validação
    if final_vector.shape[1] != 37:
        raise ValueError(f"Dimensão do vetor inválida: {final_vector.shape[1]}. Esperado: 37 features.")

    return final_vector

def get_model_decision(alert_data_dict: dict) -> dict:
    """
    Integra o pré-processamento com a previsão e a regra de negócio.
    Esta função simula o endpoint final da API.
    """

    # Threshold de 0.75 para alta Precision (regra de negócio)
    THRESHOLD_NEGOCIO = 0.75

    # --- 1. Carregar o modelo ---
    # A API idealmente faria isso APENAS uma vez na inicialização,
    # mas fazemos aqui para o teste:
    try:
        loaded_model = joblib.load("model_artifacts_deploy/xgb_champion_final.pkl")
    except FileNotFoundError:
        return {"error": "Arquivo do modelo não encontrado. Verifique o caminho."}, 500

    # 2. Pré-processar o alerta para obter o vetor de features
    model_input_vector = prepare_alert_for_model(alert_data_dict)

    # 3. Prever a probabilidade usando o modelo carregado
    # O .predict_proba retorna a probabilidade da classe 0 e 1; pegamos apenas a classe 1 [:, 1]
    probability = loaded_model.predict_proba(model_input_vector)[:, 1][0]

    # 4. Aplicar a Regra de Negócio (Threshold)
    is_incident = probability >= THRESHOLD_NEGOCIO

    # 5. Retorno formatado para a API
    return {
        "is_incident": bool(is_incident),
        "probability": float(probability),
        "decision_threshold": THRESHOLD_NEGOCIO
    }

# --- Teste Final de Simulação ---

# Alerta de Teste (Exemplo de Alto Risco)
alert_exemplo_risco = {
    'source': 'Dynatrace',
    'environment': 'prod',
    'severity': 'critical',
    'metric_name': 'connection_errors',
    'ci': 'app-7',  # O pré-processamento transforma isso em 'app'
    'maintenance': 'False'
}

# Test

In [82]:
cenario_alto_risco = {
  "source": "Dynatrace",
  "environment": "prod",
  "severity": "major",
  "metric_name": "service_unavailable",
  "ci": "db-prod-02",
  "maintenance": "false"
}

try:
    model_input_vector = prepare_alert_for_model(cenario_alto_risco)

    print("\n--- Resultado da Transformação (Etapa 5 Replicada) ---")
    print(f"Alerta de Teste: {cenario_alto_risco['severity']} em {cenario_alto_risco['environment']}")
    print(f"Shape do Vetor de Features: {model_input_vector.shape}")
    print("-" * 40)
    print("Vetor de Input Final:")
    # Imprime os valores do vetor para confirmar que eles são 0s e 1s
    print(model_input_vector[0, :37])

except ValueError as e:
    print(f"ERRO DE VALIDAÇÃO: {e}")
except NameError:
    print("ERRO: Certifique-se de que a função 'prepare_alert_for_model' e as variáveis globais foram executadas anteriormente.")


--- Resultado da Transformação (Etapa 5 Replicada) ---
Alerta de Teste: major em prod
Shape do Vetor de Features: (1, 37)
----------------------------------------
Vetor de Input Final:
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5 stored elements and shape (1, 37)>
  Coords	Values
  (0, 0)	1.0
  (0, 4)	1.0
  (0, 11)	1.0
  (0, 23)	1.0
  (0, 32)	1.0


In [83]:
# Obter a decisão
decisao_final = get_model_decision(cenario_alto_risco)

print(f"Alerta de Entrada: {cenario_alto_risco['severity']} em {cenario_alto_risco['environment']}")
print("-" * 40)
print(f"Decisão do Modelo (Final): {decisao_final}")

Alerta de Entrada: major em prod
----------------------------------------
Decisão do Modelo (Final): {'is_incident': True, 'probability': 0.9200032353401184, 'decision_threshold': 0.75}


In [84]:
cenario_baixo_risco = {
  "source": "Zabbix",
  "environment": "dev",
  "severity": "ok",
  "metric_name": "ping_failed",
  "ci": "server-test-01",
  "maintenance": "true"
}

try:
    model_input_vector = prepare_alert_for_model(cenario_baixo_risco)

    print("\n--- Resultado da Transformação (Etapa 5 Replicada) ---")
    print(f"Alerta de Teste: {cenario_baixo_risco['severity']} em {cenario_baixo_risco['environment']}")
    print(f"Shape do Vetor de Features: {model_input_vector.shape}")
    print("-" * 40)
    print("Vetor de Input Final:")
    # Imprime os valores do vetor para confirmar que eles são 0s e 1s
    print(model_input_vector[0, :37])

except ValueError as e:
    print(f"ERRO DE VALIDAÇÃO: {e}")
except NameError:
    print("ERRO: Certifique-se de que a função 'prepare_alert_for_model' e as variáveis globais foram executadas anteriormente.")


--- Resultado da Transformação (Etapa 5 Replicada) ---
Alerta de Teste: ok em dev
Shape do Vetor de Features: (1, 37)
----------------------------------------
Vetor de Input Final:
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (1, 37)>
  Coords	Values
  (0, 1)	1.0
  (0, 6)	1.0
  (0, 8)	1.0
  (0, 22)	1.0
  (0, 30)	1.0
  (0, 36)	1.0


In [85]:
# Obter a decisão
decisao_final = get_model_decision(cenario_baixo_risco)

print(f"Alerta de Entrada: {cenario_baixo_risco['severity']} em {cenario_baixo_risco['environment']}")
print("-" * 40)
print(f"Decisão do Modelo (Final): {decisao_final}")

Alerta de Entrada: ok em dev
----------------------------------------
Decisão do Modelo (Final): {'is_incident': False, 'probability': 0.02410030923783779, 'decision_threshold': 0.75}
